In [1]:
!pip install -Uqq rfdetr pybboxes supervision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 13.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.6/131.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.9/757.9 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import os
import cv2
import pandas as pd
import numpy as np
import wandb
import pybboxes as pbx
import pybboxes.functional as pbf
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageOps, ImageFont
from pathlib import Path
import supervision as sv
from tqdm.auto import tqdm
from rfdetr import RFDETRBase, RFDETRLarge
from pprint import pprint

2025-05-04 10:01:00.157285: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746352860.371374      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746352860.430822      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
#replace with your dataset paths
test_images_path = Path('/kaggle/input/amini-cocoa-contamination-challenge/Amini Cocoa Contamination Challenge/images/test')
test = pd.read_csv('/kaggle/input/amini-cocoa-contamination-challenge/Amini Cocoa Contamination Challenge/Test.csv')
ss = pd.read_csv('/kaggle/input/amini-cocoa-contamination-challenge/Amini Cocoa Contamination Challenge/SampleSubmission.csv')

In [4]:
test.head()

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id,ImagePath
0,ID_Genxyu.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dataset/images/test/ID_Genxyu.jpg
1,ID_svY6TG.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dataset/images/test/ID_svY6TG.jpg
2,ID_d0gpda.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dataset/images/test/ID_d0gpda.jpg
3,ID_frWmBT.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dataset/images/test/ID_frWmBT.jpg
4,ID_TaRW6o.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dataset/images/test/ID_TaRW6o.jpg


In [5]:
ss.head()

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
0,ID_Genxyu.jpg,healthy,0.5,100,100,100,100
1,ID_svY6TG.jpg,healthy,0.5,100,100,100,100
2,ID_d0gpda.jpg,healthy,0.5,100,100,100,100
3,ID_frWmBT.jpg,healthy,0.5,100,100,100,100
4,ID_TaRW6o.jpg,healthy,0.5,100,100,100,100


In [ ]:
#replace with your dataset paths
test['ImagePath'] = test['ImagePath'].str.replace('dataset/images/test', str(test_images_path))

In [7]:
test.head()

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id,ImagePath
0,ID_Genxyu.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/kaggle/input/amini-cocoa-contamination-challe...
1,ID_svY6TG.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/kaggle/input/amini-cocoa-contamination-challe...
2,ID_d0gpda.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/kaggle/input/amini-cocoa-contamination-challe...
3,ID_frWmBT.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/kaggle/input/amini-cocoa-contamination-challe...
4,ID_TaRW6o.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/kaggle/input/amini-cocoa-contamination-challe...


In [ ]:
annotation_labels = [
        "anthracnose",
        "cssvd",
        "healthy"
    ]


def run_inference(model_path, threshold = 0.02, n_epochs = 5):
    #ckpt_paths = [i for i in os.listdir(model_path) if i.endswith('.pth')]
    ckpt_paths  = ['checkpoint_best_ema.pth']

    for ckpt_path in tqdm(ckpt_paths, total = len(ckpt_paths)):
        if 'best' in str(ckpt_path):
            name = str(ckpt_path)[str(ckpt_path).find('best'):].split('.')[0]
        else:
            name = ckpt_path
        base_model =  RFDETRLarge(pretrain_weights = str(model_path / ckpt_path), fp16_eval = True)
        results_df = pd.DataFrame(columns=test.columns.tolist())
                                  
        for _, row in tqdm(test.iterrows(), total = len(test)):
            temp_df = pd.DataFrame(columns=test.columns.tolist())
            image = Image.open(row['ImagePath'])
            image = ImageOps.exif_transpose(image)
            sv_result = base_model.predict(image, threshold = threshold)
        
            if len(sv_result.confidence) > 0:
                xyxys = sv_result.xyxy
                confidences = sv_result.confidence
                temp_df['confidence'] = confidences
                temp_df['class_id'] = sv_result.class_id
                temp_df['class'] = [annotation_labels[i] for i in sv_result.class_id]
                temp_df[['xmin', 'ymin', 'xmax', 'ymax']] = xyxys
                temp_df['Image_ID'] = row['Image_ID']
            else:  # No detections
                print(row['Image_ID'])
                temp_df['confidence'] = [0]
                temp_df['class'] = ['healthy']
                temp_df['class_id'] = [0]
                temp_df[['xmin', 'ymin', 'xmax', 'ymax']] = [0, 0, 0, 0]
                temp_df['Image_ID'] = row['Image_ID']
        
            results_df = pd.concat([results_df, temp_df], ignore_index=True)
        results_df.to_csv(f'rfdetr large {n_epochs} epochs {threshold} conf {name}.csv', index = False)

In [ ]:
#replace with your model path
ckpt_path =  Path('/kaggle/input/rf-detr-large-amini-coco-detection/pytorch/5-epoch-baseline/4/output')

In [ ]:
run_inference(ckpt_path, threshold = 0.02, n_epochs = '9 epochs full linear')
#use this output for wbf ensembling

  0%|          | 0/2 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Loading pretrain weights


  0%|          | 0/1626 [00:00<?, ?it/s]

Loading pretrain weights


  0%|          | 0/1626 [00:00<?, ?it/s]